# Training With Autoregressive

In [1]:
N_THREADS = 8
# Nota Bene : notebooks don't deallocate GPU memory
IS_FORCE_CPU = False # can also be set in the trainer

In [2]:
cd ..

/private/home/yannd/projects/Neural-Process-Family


In [3]:
%autosave 600
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# CENTER PLOTS
from IPython.core.display import HTML
display(HTML(""" <style> .output_png {display: table-cell; text-align: center; margin:auto; }
.prompt display:none;}  </style>"""))

import os
if IS_FORCE_CPU:
    os.environ['CUDA_VISIBLE_DEVICES'] = ""
    
import sys
sys.path.append("notebooks")

import numpy as np
import matplotlib.pyplot as plt
import torch
torch.set_num_threads(N_THREADS)

Autosaving every 600 seconds


In [4]:
import neuralproc
from train_imgs import main, parse_arguments

In [5]:
from utils.data import get_dataset
from utils.data.helpers import train_dev_split

celeba32_train, celeba32_test = train_dev_split(get_dataset("celeba32")(), dev_size=0.1, is_stratify=False)
train_datasets = {"celeba32":celeba32_train}
test_datasets = {"celeba32":celeba32_test}

In [6]:
from neuralproc.utils.datasplit import GridCntxtTrgtGetter, RandomMasker, no_masker, half_masker

# same as in previous tutorials but uses masks rather than indices
get_cntxt_trgt = GridCntxtTrgtGetter(context_masker=RandomMasker(min_nnz=0.01, max_nnz=0.5, is_batch_share=True),
                                      target_masker=no_masker,
                                      is_add_cntxts_to_trgts=False)

In [8]:
X_DIM = 2  # 2D spatial input 
import torch.nn as nn
from functools import partial
from neuralproc import RegularGridsConvolutionalProcess, AttentiveNeuralProcess, NeuralProcessLoss, ConvolutionalProcess
from neuralproc.predefined import UnetCNN, CNN, SelfAttention, MLP, ResConvBlock
from neuralproc import merge_flat_input
from neuralproc.utils.predict import GenAllAutoregressivePixel

from neuralproc.utils.helpers import (MultivariateNormalDiag, ProbabilityConverter,
                                      make_abs_conv)

kwargs = dict(r_dim=128,
              # make sure output is in 0,1 as images preprocessed so
              pred_loc_transformer=lambda mu: torch.sigmoid(mu))

# models that take into account the grid
models_grided = {}
models_grided["test"] = partial(RegularGridsConvolutionalProcess, 
                                     x_dim=X_DIM,
                                     Conv=lambda y_dim: make_abs_conv(nn.Conv2d)(
                                                y_dim,
                                                y_dim,
                                                groups=y_dim,
                                                kernel_size=7,
                                                padding=7 // 2,
                                                bias=False,
                                            ),
                                     PseudoTransformer=partial(CNN, 
                                                             ConvBlock=ResConvBlock,
                                                             Conv=torch.nn.Conv2d,
                                                             n_blocks=5,  
                                                             Normalization=torch.nn.BatchNorm2d, 
                                                             is_chan_last=True,
                                                             kernel_size=3),
                                        n_autoregressive_steps=3,
                                        is_autoregress_confidence=True,
                                         get_gen_autoregressive_trgts=GenAllAutoregressivePixel(),
                                     **kwargs)


In [9]:
from utils.helpers import count_parameters
    
for k,v in models_grided.items():
    print(k, "- N Param:", count_parameters(v(y_dim=3)))

GridedCCP_ARC - N Param: 166561


In [10]:
def add_y_dim(models, datasets):
    """Add y _dim to all ofthe models depending on the dataset."""
    return {data_name: {model_name: partial(model, y_dim=data_train.shape[0]) 
                        for model_name, model in models.items()} 
            for data_name, data_train in datasets.items()}

In [11]:
from neuralproc import NeuralProcessLoss
from utils.train import train_models
from utils.data import cntxt_trgt_collate
import skorch

In [ ]:
# still training celeba64
trainers_grided = train_models(train_datasets, 
                                 add_y_dim(models_grided, train_datasets),
                                 NeuralProcessLoss,
                                 test_datasets=test_datasets,
                                 chckpnt_dirname="results/imgs/", 
                                 is_retrain=True, 
                                 train_split=skorch.dataset.CVSplit(0.1), # use 10% of data for validation 
                                 iterator_train__collate_fn=cntxt_trgt_collate(get_cntxt_trgt, 
                                                                               is_return_masks=True), 
                                 iterator_valid__collate_fn=cntxt_trgt_collate(get_cntxt_trgt, 
                                                                               is_return_masks=True),
                                 patience=10,
                                 seed=123) 


--- Training celeba32/GridedCCP_ARC/run_0 ---



In [ ]:
import torch

In [ ]:
t=torch.BoolTensor([True,False])

In [ ]:
t[:]=1

In [ ]:
t